In [11]:
import tensorflow as tf
import os, shutil
from matplotlib import pyplot as plt
# import datasets 
import numpy as np
import os
from keras import layers
from keras import models
import pathlib
import PIL.Image #abbreviation of python Image Library, a.k.a Pillow) import Image to load, manage, save image files in diverse formats
import IPython.display as display #to display images later on
from keras.preprocessing.image import ImageDataGenerator


In [6]:
# Change train_dir, validation_dir and test_dir (AT LEAST) to match the location of such folders in your machine
original_dataset_dir = '/home/iris/Downloads/original_dataset/train/train'
base_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small'
train_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/train'
validation_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/validation'
test_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/test'
train_cats_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/train/cats'
train_dogs_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/train/dogs'
validation_cats_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/validation/cats'
validation_dogs_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/validation/dogs'
test_cats_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/test/cats'
test_dogs_dir = '/home/iris/catkin_ws/src/tensorflow_ex/datasets/cats_and_dogs_small/test/dogs'

fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


#separate train and test data
train = list(train_dir.glob('dogs/*'))
PIL.Image.open(str(train[0]))
test = list(test_dir.glob('test1/*'))
PIL.Image.open(str(test[0]))

### SIMPLE MODEL CREATION

In [12]:
num_classes = 2
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
    input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [14]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-4),
metrics=['acc'])

In [16]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [17]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [18]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
100/100 [==============================] - 72s 718ms/step - loss: 0.6935 - acc: 0.5115 - val_loss: 0.6726 - val_acc: 0.6110
Epoch 2/30
100/100 [==============================] - 68s 685ms/step - loss: 0.6641 - acc: 0.5950 - val_loss: 0.6419 - val_acc: 0.6310
Epoch 3/30
100/100 [==============================] - 69s 690ms/step - loss: 0.6245 - acc: 0.6605 - val_loss: 0.6130 - val_acc: 0.6520
Epoch 4/30
100/100 [==============================] - 70s 695ms/step - loss: 0.5749 - acc: 0.6970 - val_loss: 0.5865 - val_acc: 0.6740
Epoch 5/30
100/100 [==============================] - 79s 792ms/step - loss: 0.5391 - acc: 0.7300 - val_loss: 0.5843 - val_acc: 0.6820
Epoch 6/30
100/100 [==============================] - 65s 649ms/step - loss: 0.5185 - acc: 0.7435 - val_loss: 0.5812 - val_acc: 0.6870
Epoch 7/30
100/100 [==============================] - 64s 636ms/step - loss: 0.4867 - acc: 0.7695 - val_loss: 0.62

KeyboardInterrupt: 

In [ ]:
model.save('cats_and_dogs_small_1.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()